In [ ]:
!pip install torch==1.4.0 torchvision==0.5.0

     |████████████████████████████████| 753.4 MB 7.1 kB/s 
     |████████████████████████████████| 4.0 MB 33.0 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.4.0 which is incompatible.


In [ ]:
# 乱数のシードを設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)

# 使用するパッケージ（ライブラリと関数）を定義
# 標準正規分布の生成用
from numpy.random import *

# グラフの描画用
import matplotlib.pyplot as plt

# その他
import pandas as pd

# シグモイド関数をimport
from scipy.special import expit

In [ ]:
df = pd.read_csv('/content/コロナデータ　実験２.csv')

In [ ]:
df.head()

,感染者数,飲食店情報閲覧\n（前年同週比を日毎データに）,人流データ 小売店と娯楽施設\n（基準：2020 年 1 月 3 日〜2 月 6 日の 5 週間の曜日別中央値）,食料品店と薬局,公共交通機関,実効再生産数,緊急事態宣言\n（宣言中は１）,天気\n（全国的に半分が雨\nor台風など上陸などがあったら１）
0,4,-4.310739,-1,4,1,0.749,0,1
1,1,-4.310739,-9,-6,-10,1.241,0,1
2,6,-4.310739,-2,1,-1,1.915,0,1
3,8,-4.310739,1,2,-2,2.692,0,0
4,9,-4.310739,0,2,-3,4.116,0,0


In [ ]:
df.isnull().sum()

感染者数                                                        0
飲食店情報閲覧\n（前年同週比を日毎データに）                                     0
人流データ　小売店と娯楽施設\n（基準：2020 年 1 月 3 日〜2 月 6 日の 5 週間の曜日別中央値）    0
食料品店と薬局                                                     0
公共交通機関                                                      0
実効再生産数                                                      0
緊急事態宣言\n（宣言中は１）                                             0
天気\n（全国的に半分が雨\nor台風など上陸などがあったら１）                            0
dtype: int64

In [ ]:
!pip install cdt==0.5.18

     |████████████████████████████████| 917 kB 5.2 MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=2a309a474fadc664238d63e889bdcdf86ed4cf8a000816a77ab9d389f54700f9
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
  Created wheel for skrebate: filename=skrebate-0.62-py3-none-any.whl size=29269 sha256=d1c660da4c758692e7a47be3b984ddaf3dfa6c23b0691262b75e5442c5611fde
  Stored in directory: /root/.cache/pip/wheels/4f/3b/07/22a9a01f3f25cd376122b5ef526737a44993b4c312e50916ed
Successfully built GPUtil skrebate


In [ ]:
# PyTorchから使用するものをimport
import torch
import torch.nn as nn


class SAMDiscriminator(nn.Module):
    """SAMのDiscriminatorのニューラルネットワーク
    """

    def __init__(self, nfeatures, dnh, hlayers):
        super(SAMDiscriminator, self).__init__()

        # ----------------------------------
        # ネットワークの用意
        # ----------------------------------
        self.nfeatures = nfeatures  # 入力変数の数

        layers = []
        layers.append(nn.Linear(nfeatures, dnh))
        layers.append(nn.BatchNorm1d(dnh))
        layers.append(nn.LeakyReLU(.2))

        for i in range(hlayers-1):
            layers.append(nn.Linear(dnh, dnh))
            layers.append(nn.BatchNorm1d(dnh))
            layers.append(nn.LeakyReLU(.2))

        layers.append(nn.Linear(dnh, 1))  # 最終出力

        self.layers = nn.Sequential(*layers)

        # ----------------------------------
        # maskの用意（対角成分のみ1で、他は0の行列）
        # ----------------------------------
        mask = torch.eye(nfeatures, nfeatures)  # 変数の数×変数の数の単位行列
        self.register_buffer("mask", mask.unsqueeze(0))  # 単位行列maskを保存しておく

        # 注意：register_bufferはmodelのパラメータではないが、その後forwardで使う変数を登録するPyTorchのメソッドです
        # self.変数名で、以降も使用可能になります
        # https://pytorch.org/docs/stable/nn.html?highlight=register_buffer#torch.nn.Module.register_buffer

    def forward(self, input, obs_data=None):
        """　順伝搬の計算
        Args:
            input (torch.Size([データ数, 観測変数の種類数])): 観測したデータ、もしくは生成されたデータ
            obs_data (torch.Size([データ数, 観測変数の種類数])):観測したデータ
        Returns:
            torch.Tensor: 観測したデータか、それとも生成されたデータかの判定結果
        """

        if obs_data is not None:
          # 生成データを識別器に入力する場合
            return [self.layers(i) for i in torch.unbind(obs_data.unsqueeze(1) * (1 - self.mask)
                                                         + input.unsqueeze(1) * self.mask, 1)]
            # 対角成分のみ生成したデータ、その他は観測データに
            # データを各変数ごとに、生成したもの、その他観測したもので混ぜて、1変数ずつ生成したものを放り込む
            # torch.unbind(x,1)はxの1次元目でテンソルをタプルに展開する
            # minibatch数が2000、観測データの変数が6種類の場合、
            # [2000,6]→[2000,6,6]→([2000,6], [2000,6], [2000,6], [2000,6], [2000,6], [2000,6])→([2000,1], [2000,1], [2000,1], [2000,1], [2000,1], [2000,1])
            # returnは[torch.Size([2000, 1]),torch.Size([2000, 1]),torch.Size([2000, 1], torch.Size([2000, 1]),torch.Size([2000, 1]),torch.Size([2000, 1])]

            # 注：生成した変数全種類を用いた判定はしない。
            # すなわち、生成した変数1種類と、元の観測データたちをまとめて1つにし、それが観測結果か、生成結果を判定させる

        else:
            # 観測データを識別器に入力する場合

            return self.layers(input)
            # returnは[torch.Size([2000, 1])]


    def reset_parameters(self):
        """識別器Dの重みパラメータの初期化を実施"""
        for layer in self.layers:
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()

In [ ]:
from cdt.utils.torch import ChannelBatchNorm1d, MatrixSampler, Linear3D


class SAMGenerator(nn.Module):
    """SAMのGeneratorのニューラルネットワーク
    """

    def __init__(self, data_shape, nh):
        """初期化"""
        super(SAMGenerator, self).__init__()

        # ----------------------------------
        # 対角成分のみ0で、残りは1のmaskとなる変数skeletonを作成
        # ※最後の行は、全部1です
        # ----------------------------------
        nb_vars = data_shape[1]  # 変数の数
        skeleton = 1 - torch.eye(nb_vars + 1, nb_vars)

        self.register_buffer('skeleton', skeleton)

        # 注意：register_bufferはmodelのパラメータではないが、その後forwardで使う変数を登録するPyTorchのメソッドです
        # self.変数名で、以降も使用可能になります
        # https://pytorch.org/docs/stable/nn.html?highlight=register_buffer#torch.nn.Module.register_buffer

        # ----------------------------------
        # ネットワークの用意
        # ----------------------------------
        # 入力層（SAMの形での全結合層）　
        self.input_layer = Linear3D(
            (nb_vars, nb_vars + 1, nh))  # nhは中間層のニューロン数
        # https://github.com/FenTechSolutions/CausalDiscoveryToolbox/blob/32200779ab9b63762be3a24a2147cff09ba2bb72/cdt/utils/torch.py#L289

        # 中間層
        layers = []
        # 2次元を1次元に変換してバッチノーマライゼーションするモジュール
        layers.append(ChannelBatchNorm1d(nb_vars, nh))
        layers.append(nn.Tanh())
        self.layers = nn.Sequential(*layers)

        # ChannelBatchNorm1d
        # https://github.com/FenTechSolutions/CausalDiscoveryToolbox/blob/32200779ab9b63762be3a24a2147cff09ba2bb72/cdt/utils/torch.py#L130

        # 出力層（再度、SAMの形での全結合層）
        self.output_layer = Linear3D((nb_vars, nh, 1))

    def forward(self, data, noise, adj_matrix, drawn_neurons=None):
        """　順伝搬の計算
        Args:
            data (torch.Tensor): 観測データ
            noise (torch.Tensor): データ生成用のノイズ
            adj_matrix (torch.Tensor): 因果関係を示す因果構造マトリクスM
            drawn_neurons (torch.Tensor): Linear3Dの複雑さを制御する複雑さマトリクスZ
        Returns:
            torch.Tensor: 生成されたデータ
        """

        # 入力層
        x = self.input_layer(data, noise, adj_matrix *
                             self.skeleton)  # Linear3D

        # 中間層（バッチノーマライゼーションとTanh）
        x = self.layers(x)

        # 出力層
        output = self.output_layer(
            x, noise=None, adj_matrix=drawn_neurons)  # Linear3D

        return output.squeeze(2)

    def reset_parameters(self):
        """重みパラメータの初期化を実施"""

        self.input_layer.reset_parameters()
        self.output_layer.reset_parameters()

        for layer in self.layers:
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()

Detecting 1 CUDA device(s).


In [ ]:
# ネットワークを示す因果構造マトリクスMがDAG（有向非循環グラフ）になるように加える損失

def notears_constr(adj_m, max_pow=None):
    """No Tears constraint for binary adjacency matrixes. 
    Args:
        adj_m (array-like): Adjacency matrix of the graph
        max_pow (int): maximum value to which the infinite sum is to be computed.
           defaults to the shape of the adjacency_matrix
    Returns:
        np.ndarray or torch.Tensor: Scalar value of the loss with the type
            depending on the input.
    参考：https://github.com/FenTechSolutions/CausalDiscoveryToolbox/blob/32200779ab9b63762be3a24a2147cff09ba2bb72/cdt/utils/loss.py#L215
    """
    m_exp = [adj_m]
    if max_pow is None:
        max_pow = adj_m.shape[1]
    while(m_exp[-1].sum() > 0 and len(m_exp) < max_pow):
        m_exp.append(m_exp[-1] @ adj_m/len(m_exp))

    return sum([i.diag().sum() for idx, i in enumerate(m_exp)])

In [ ]:
from sklearn.preprocessing import scale
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm


def run_SAM(in_data, lr_gen, lr_disc, lambda1, lambda2, hlayers, nh, dnh, train_epochs, test_epochs, device):
    '''SAMの学習を実行する関数'''

    # ---------------------------------------------------
    # 入力データの前処理
    # ---------------------------------------------------
    list_nodes = list(in_data.columns)  # 入力データの列名のリスト
    data = scale(in_data[list_nodes].values)  # 入力データの正規化
    nb_var = len(list_nodes)  # 入力データの数 = d
    data = data.astype('float32')  # 入力データをfloat32型に
    data = torch.from_numpy(data).to(device)  # 入力データをPyTorchのテンソルに
    rows, cols = data.size()  # rowsはデータ数、colsは変数の数

    # ---------------------------------------------------
    # DataLoaderの作成（バッチサイズは全データ）
    # ---------------------------------------------------
    batch_size = rows  # 入力データ全てを使用したミニバッチ学習とする
    data_iterator = DataLoader(data, batch_size=batch_size,
                               shuffle=True, drop_last=True)
    # 注意：引数のdrop_lastはdataをbatch_sizeで取り出していったときに最後に余ったものは使用しない設定

    # ---------------------------------------------------
    # 【Generator】ネットワークの生成とパラメータの初期化
    # cols：入力変数の数、nhは中間ニューロンの数、hlayersは中間層の数
    # neuron_samplerは、Functional gatesの変数zを学習するネットワーク
    # graph_samplerは、Structual gatesの変数aを学習するネットワーク
    # ---------------------------------------------------
    sam = SAMGenerator((batch_size, cols), nh).to(device)  # 生成器G
    graph_sampler = MatrixSampler(nb_var, mask=None, gumbel=False).to(
        device)  # 因果構造マトリクスMを作るネットワーク
    neuron_sampler = MatrixSampler((nh, nb_var), mask=False, gumbel=True).to(
        device)  # 複雑さマトリクスZを作るネットワーク

    # 注意：MatrixSamplerはGumbel-Softmaxを使用し、0か1を出力させるニューラルネットワーク
    # SAMの著者らの実装モジュール、MatrixSamplerを使用
    # https://github.com/FenTechSolutions/CausalDiscoveryToolbox/blob/32200779ab9b63762be3a24a2147cff09ba2bb72/cdt/utils/torch.py#L212

    # 重みパラメータの初期化
    sam.reset_parameters()
    graph_sampler.weights.data.fill_(2)

    # ---------------------------------------------------
    # 【Discriminator】ネットワークの生成とパラメータの初期化
    # cols：入力変数の数、dnhは中間ニューロンの数、hlayersは中間層の数。
    # ---------------------------------------------------
    discriminator = SAMDiscriminator(cols, dnh, hlayers).to(device)
    discriminator.reset_parameters()  # 重みパラメータの初期化

    # ---------------------------------------------------
    # 最適化の設定
    # ---------------------------------------------------
    # 生成器

    g_optimizer = optim.Adam(sam.parameters(), lr=lr_gen)
    graph_optimizer = optim.Adam(graph_sampler.parameters(), lr=lr_gen)
    neuron_optimizer = optim.Adam(neuron_sampler.parameters(), lr=lr_gen)

    # 識別器
    d_optimizer = optim.Adam(discriminator.parameters(), lr=lr_disc)

    # 損失関数
    criterion = nn.BCEWithLogitsLoss()
    # nn.BCEWithLogitsLoss()は、binary cross entropy with Logistic function
    # https://pytorch.org/docs/stable/nn.html#bcewithlogitsloss

    # 損失関数のDAGに関する制約の設定パラメータ
    dagstart = 0.5
    dagpenalization_increase = 0.001*10

    # ---------------------------------------------------
    # forward計算、および損失関数の計算に使用する変数を用意
    # ---------------------------------------------------
    _true = torch.ones(1).to(device)
    _false = torch.zeros(1).to(device)

    noise = torch.randn(batch_size, nb_var).to(device)  # 生成器Gで使用する生成ノイズ
    noise_row = torch.ones(1, nb_var).to(device)

    output = torch.zeros(nb_var, nb_var).to(device)  # 求まった隣接行列
    output_loss = torch.zeros(1, 1).to(device)

    # ---------------------------------------------------
    # forwardの計算で、ネットワークを学習させる
    # ---------------------------------------------------
    pbar = tqdm(range(train_epochs + test_epochs),mininterval=1)  # 進捗（progressive bar）の表示

    for epoch in pbar:
        for i_batch, batch in enumerate(data_iterator):

            # 最適化を初期化
            g_optimizer.zero_grad()
            graph_optimizer.zero_grad()
            neuron_optimizer.zero_grad()
            d_optimizer.zero_grad()

            # 因果構造マトリクスM（drawn_graph）と複雑さマトリクスZ（drawn_neurons）をMatrixSamplerから取得
            drawn_graph = graph_sampler()
            drawn_neurons = neuron_sampler()
            # (drawn_graph)のサイズは、torch.Size([nb_var, nb_var])。 出力値は0か1
            # (drawn_neurons)のサイズは、torch.Size([nh, nb_var])。 出力値は0か1

            # ノイズをリセットし、生成器Gで疑似データを生成
            noise.normal_()
            generated_variables = sam(data=batch, noise=noise,
                                      adj_matrix=torch.cat(
                                          [drawn_graph, noise_row], 0),
                                      drawn_neurons=drawn_neurons)

            # 識別器Dで判定
            # 観測変数のリスト[]で、各torch.Size([data数, 1])が求まる
            disc_vars_d = discriminator(generated_variables.detach(), batch)
            # 観測変数のリスト[] で、各torch.Size([data数, 1])が求まる
            disc_vars_g = discriminator(generated_variables, batch)
            true_vars_disc = discriminator(batch)  # torch.Size([data数, 1])が求まる

            # 損失関数の計算（DCGAN）
            disc_loss = sum([criterion(gen, _false.expand_as(gen)) for gen in disc_vars_d]) / nb_var \
                + criterion(true_vars_disc, _true.expand_as(true_vars_disc))

            gen_loss = sum([criterion(gen,
                                      _true.expand_as(gen))
                            for gen in disc_vars_g])

            # 損失の計算（SAM論文のオリジナルのfgan）
            #disc_loss = sum([torch.mean(torch.exp(gen - 1)) for gen in disc_vars_d]) / nb_var - torch.mean(true_vars_disc)
            #gen_loss = -sum([torch.mean(torch.exp(gen - 1)) for gen in disc_vars_g])

            # 識別器Dのバックプロパゲーションとパラメータの更新
            if epoch < train_epochs:
                disc_loss.backward()
                d_optimizer.step()

            # 生成器のGの損失の計算の残り（マトリクスの複雑さとDAGのNO TEAR）
            struc_loss = lambda1 / batch_size*drawn_graph.sum()     # Mのloss
            func_loss = lambda2 / batch_size*drawn_neurons.sum()   # Aのloss

            regul_loss = struc_loss + func_loss

            if epoch <= train_epochs * dagstart:
                # epochが基準前のときは、DAGになるようにMへのNO TEARSの制限はかけない
                loss = gen_loss + regul_loss

            else:
                # epochが基準後のときは、DAGになるようにNO TEARSの制限をかける
                filters = graph_sampler.get_proba()  # マトリクスMの要素を取得（ただし、0,1ではなく、1の確率）
                dag_constraint = notears_constr(filters*filters)  # NO TERARの計算

                # 徐々に線形にDAGの正則を強くする
                loss = gen_loss + regul_loss + \
                    ((epoch - train_epochs * dagstart) *
                     dagpenalization_increase) * dag_constraint

            if epoch >= train_epochs:
                # testのepochの場合、結果を取得
                output.add_(filters.data)
                output_loss.add_(gen_loss.data)
            else:
                # trainのepochの場合、生成器Gのバックプロパゲーションと更新
                # retain_graph=Trueにすることで、以降3つのstep()が実行できる
                loss.backward(retain_graph=True)
                g_optimizer.step()
                graph_optimizer.step()
                neuron_optimizer.step()

            # 進捗の表示
            if epoch % 50 == 0:
                pbar.set_postfix(gen=gen_loss.item()/cols,
                                 disc=disc_loss.item(),
                                 regul_loss=regul_loss.item(),
                                 tot=loss.item())

    return output.cpu().numpy()/test_epochs, output_loss.cpu().numpy()/test_epochs/cols  # Mと損失を出力


In [ ]:
# GPUの使用確認：True or False
torch.cuda.is_available()

True

学習率をいじる　0.5を0.05

In [ ]:
# numpyの出力を小数点2桁に
np.set_printoptions(precision=2, floatmode='fixed', suppress=True)

# 因果探索の結果を格納するリスト
m_list = []
loss_list = []

for i in range(5):
    m, loss = run_SAM(in_data=df, lr_gen=0.01*0.5,
                      lr_disc=0.01*0.5*2,
                      #lambda1=0.01, lambda2=1e-05,
                      lambda1=5.0*20, lambda2=0.005*20,
                      hlayers=2,
                      nh=200, dnh=200,
                      train_epochs=10000,
                      test_epochs=1000,
                      device='cuda:0')

    print(loss)
    print(m)

    m_list.append(m)
    loss_list.append(loss)

100%|██████████| 11000/11000 [06:38<00:00, 27.58it/s, disc=0.129, gen=8.23, regul_loss=3.46, tot=71.2]


[[7.94]]
[[0.00 0.95 0.41 0.01 0.00 1.00 0.02 0.03]
 [0.04 0.00 0.05 0.01 0.03 0.03 1.00 0.00]
 [0.00 0.99 0.00 0.01 0.04 0.84 0.35 0.00]
 [0.99 0.97 0.98 0.00 0.99 0.94 0.00 0.00]
 [0.30 0.95 0.84 0.00 0.00 0.96 0.00 0.00]
 [0.01 0.32 0.00 0.00 0.00 0.00 0.96 0.00]
 [0.04 0.02 0.02 0.01 0.02 0.04 0.00 0.00]
 [0.99 0.95 0.96 0.99 0.00 0.98 0.00 0.00]]


100%|██████████| 11000/11000 [06:38<00:00, 27.61it/s, disc=0.199, gen=7.78, regul_loss=3.79, tot=68.4]


[[8.06]]
[[0.00 0.02 0.08 0.00 0.81 1.00 0.95 0.00]
 [0.99 0.00 0.09 0.02 0.98 0.59 0.99 0.00]
 [0.00 0.95 0.00 0.05 0.99 0.99 0.00 0.00]
 [0.99 0.78 0.99 0.00 0.98 0.99 0.01 0.00]
 [0.01 0.03 0.04 0.00 0.00 0.98 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.01 0.00 0.01 0.00]
 [0.01 0.02 0.02 0.00 0.43 0.58 0.00 0.00]
 [1.00 0.98 0.98 0.98 0.92 0.00 0.89 0.00]]


100%|██████████| 11000/11000 [06:42<00:00, 27.31it/s, disc=0.168, gen=8.62, regul_loss=3.11, tot=76.7]


[[8.33]]
[[0.00 0.91 0.00 0.00 0.00 0.00 0.53 0.03]
 [0.09 0.00 1.00 0.02 0.94 0.00 1.00 0.00]
 [0.00 0.04 0.00 0.02 0.00 0.99 0.00 0.00]
 [0.02 0.63 1.00 0.00 0.91 0.98 0.82 0.00]
 [0.01 0.14 0.99 0.01 0.00 0.96 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.01 0.00]
 [0.03 0.07 0.46 0.00 0.41 0.82 0.00 0.00]
 [0.99 0.94 0.99 0.99 0.96 0.98 0.00 0.00]]


100%|██████████| 11000/11000 [06:33<00:00, 27.96it/s, disc=0.0247, gen=11.3, regul_loss=3.28, tot=99.2]


[[12.26]]
[[0.00 0.03 0.00 0.00 0.44 0.98 0.41 0.01]
 [0.20 0.00 1.00 0.68 0.02 0.06 0.99 0.00]
 [0.00 0.02 0.00 0.05 0.01 0.02 0.00 0.00]
 [0.04 0.02 1.00 0.00 0.00 0.01 0.00 0.00]
 [0.02 0.96 1.00 0.04 0.00 0.06 0.00 0.07]
 [0.00 0.00 0.99 0.97 0.99 0.00 0.25 0.04]
 [0.09 0.06 0.55 0.83 0.04 0.17 0.00 0.01]
 [0.55 0.78 0.99 0.99 0.00 0.24 0.88 0.00]]


100%|██████████| 11000/11000 [06:44<00:00, 27.17it/s, disc=0.375, gen=6.79, regul_loss=3.28, tot=58.7]

[[6.75]]
[[0.00 0.93 0.00 0.00 0.98 1.00 0.69 0.01]
 [0.03 0.00 1.00 0.01 0.00 0.62 0.98 0.00]
 [0.01 0.03 0.00 0.01 0.00 0.80 0.00 0.00]
 [0.98 0.79 1.00 0.00 0.06 0.94 0.05 0.00]
 [0.00 0.98 0.99 0.00 0.00 0.96 0.00 0.00]
 [0.00 0.01 0.00 0.02 0.01 0.00 0.00 0.00]
 [0.03 0.03 0.69 0.01 0.05 0.04 0.00 0.00]
 [0.96 0.97 0.99 0.93 0.99 0.98 0.00 0.00]]


In [ ]:
final_list=sum(m_list)/len(m_list)

In [ ]:
final_list

array([[0.00, 0.57, 0.10, 0.00, 0.45, 0.80, 0.52, 0.02],
       [0.27, 0.00, 0.63, 0.15, 0.40, 0.26, 0.99, 0.00],
       [0.00, 0.41, 0.00, 0.03, 0.21, 0.73, 0.07, 0.00],
       [0.61, 0.64, 0.99, 0.00, 0.59, 0.77, 0.18, 0.00],
       [0.07, 0.61, 0.77, 0.01, 0.00, 0.78, 0.00, 0.01],
       [0.00, 0.07, 0.20, 0.20, 0.20, 0.00, 0.25, 0.01],
       [0.04, 0.04, 0.35, 0.17, 0.19, 0.33, 0.00, 0.00],
       [0.90, 0.93, 0.99, 0.98, 0.58, 0.64, 0.35, 0.00]], dtype=float32)

In [ ]:
df = pd.read_csv('/content/コロナデータ　実験３.csv')

,感染者数,飲食店情報閲覧\n（前年同週比を日毎データに）,人流データ 小売店と娯楽施設\n（基準：2020 年 1 月 3 日〜2 月 6 日の 5 週間の曜日別中央値）,食料品店と薬局,公共交通機関,実効再生産数,緊急事態宣言\n（宣言中は１）,天気\n（全国的に半分が雨\nor台風など上陸などがあったら１）
0,8.0,-4.310739,-1.0,4.0,1.0,0.749,0.0,1.0
1,22.0,-4.310739,-9.0,-6.0,-10.0,1.241,0.0,1.0
2,24.0,-4.310739,-2.0,1.0,-1.0,1.915,0.0,1.0
3,19.0,-4.310739,1.0,2.0,-2.0,2.692,0.0,0.0
4,9.0,-4.310739,0.0,2.0,-3.0,4.116,0.0,0.0
...,...,...,...,...,...,...,...,...
583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df=df[0:578]

In [ ]:
df.isnull().sum()

感染者数                                                        0
飲食店情報閲覧\n（前年同週比を日毎データに）                                     0
人流データ　小売店と娯楽施設\n（基準：2020 年 1 月 3 日〜2 月 6 日の 5 週間の曜日別中央値）    0
食料品店と薬局                                                     0
公共交通機関                                                      0
実効再生産数                                                      0
緊急事態宣言\n（宣言中は１）                                             0
天気\n（全国的に半分が雨\nor台風など上陸などがあったら１）                            0
dtype: int64

感染者数をずらす

In [ ]:
# numpyの出力を小数点2桁に
np.set_printoptions(precision=2, floatmode='fixed', suppress=True)

# 因果探索の結果を格納するリスト
m_list = []
loss_list = []

for i in range(5):
    m, loss = run_SAM(in_data=df, lr_gen=0.01*0.5,
                      lr_disc=0.01*0.5*2,
                      #lambda1=0.01, lambda2=1e-05,
                      lambda1=5.0*20, lambda2=0.005*20,
                      hlayers=2,
                      nh=200, dnh=200,
                      train_epochs=10000,
                      test_epochs=1000,
                      device='cuda:0')

    print(loss)
    print(m)

    m_list.append(m)
    loss_list.append(loss)

100%|██████████| 11000/11000 [06:45<00:00, 27.10it/s, disc=0.432, gen=3.44, regul_loss=4.03, tot=33.5]


[[4.17]]
[[0.00 0.97 0.99 0.00 0.00 0.07 0.99 0.00]
 [0.04 0.00 0.07 0.01 0.01 0.01 1.00 0.00]
 [0.00 0.01 0.00 0.03 0.99 0.00 0.00 0.01]
 [0.81 0.50 1.00 0.00 0.34 0.07 0.99 0.00]
 [0.01 0.98 0.01 0.01 0.00 0.04 0.07 0.00]
 [0.82 0.00 0.94 0.26 0.94 0.00 0.03 0.00]
 [0.02 0.01 0.05 0.00 0.03 0.02 0.00 0.01]
 [0.98 0.94 0.99 0.99 0.97 0.99 0.88 0.00]]


100%|██████████| 11000/11000 [06:47<00:00, 26.98it/s, disc=0.0254, gen=9.52, regul_loss=3.34, tot=84.8]


[[9.75]]
[[0.00 0.97 0.94 0.00 0.00 0.99 0.00 0.02]
 [0.00 0.00 0.89 1.00 0.06 0.07 0.99 0.00]
 [0.00 0.05 0.00 0.96 0.10 0.04 0.00 0.00]
 [0.01 0.00 0.08 0.00 0.02 0.00 0.05 0.00]
 [0.04 0.99 0.68 0.93 0.00 0.11 0.13 0.00]
 [0.01 0.01 0.00 1.00 0.91 0.00 0.43 0.00]
 [0.04 0.02 0.17 0.00 0.04 0.09 0.00 0.00]
 [0.92 0.98 0.70 0.98 0.97 0.85 0.07 0.00]]


100%|██████████| 11000/11000 [06:48<00:00, 26.92it/s, disc=0.00878, gen=11.3, regul_loss=3.18, tot=104]


[[10.58]]
[[0.00 0.05 0.00 0.00 0.00 0.05 0.00 0.01]
 [0.88 0.00 0.98 0.00 0.21 0.51 1.00 0.00]
 [0.95 0.09 0.00 0.98 0.66 0.89 0.00 0.01]
 [0.98 0.02 0.13 0.00 0.10 0.03 0.01 0.00]
 [0.98 0.29 0.11 0.04 0.00 0.99 0.00 0.01]
 [0.00 0.03 0.00 0.39 0.06 0.00 0.09 0.00]
 [1.00 0.06 0.07 0.98 0.29 0.20 0.00 0.01]
 [0.99 0.95 0.99 0.98 0.85 0.96 0.00 0.00]]


100%|██████████| 11000/11000 [06:52<00:00, 26.65it/s, disc=0.111, gen=8.75, regul_loss=3.68, tot=76.9]


[[9.25]]
[[0.00 0.98 0.06 0.00 0.00 1.00 0.01 0.02]
 [0.01 0.00 1.00 1.00 0.97 0.90 1.00 0.00]
 [0.02 0.02 0.00 0.98 0.00 0.00 0.00 0.01]
 [0.01 0.01 0.11 0.00 0.02 0.01 0.00 0.00]
 [0.00 0.06 1.00 0.86 0.00 0.95 0.00 0.01]
 [0.01 0.00 0.00 0.98 0.00 0.00 0.03 0.00]
 [0.02 0.06 0.95 0.98 0.85 0.75 0.00 0.00]
 [0.78 0.88 0.98 0.99 0.01 0.97 0.00 0.00]]


100%|██████████| 11000/11000 [07:05<00:00, 25.85it/s, disc=0.0932, gen=10.1, regul_loss=3.51, tot=91.2]

[[8.94]]
[[0.00 0.02 0.99 0.94 0.00 0.99 0.20 0.02]
 [0.55 0.00 0.99 1.00 0.09 0.97 0.94 0.00]
 [0.02 0.02 0.00 1.00 0.16 0.00 0.00 0.01]
 [0.02 0.02 0.06 0.00 0.01 0.06 0.00 0.00]
 [0.05 0.99 0.22 0.97 0.00 0.72 0.00 0.00]
 [0.05 0.01 0.15 0.00 0.02 0.00 0.18 0.00]
 [0.14 0.03 0.21 0.98 0.11 0.15 0.00 0.01]
 [0.99 0.99 0.87 0.97 0.11 0.48 0.00 0.00]]


In [ ]:
final_list=sum(m_list)/len(m_list)

In [ ]:
final_list

array([[0.00, 0.60, 0.60, 0.19, 0.00, 0.62, 0.24, 0.01],
       [0.29, 0.00, 0.79, 0.60, 0.27, 0.49, 0.99, 0.00],
       [0.20, 0.04, 0.00, 0.79, 0.38, 0.19, 0.00, 0.01],
       [0.37, 0.11, 0.28, 0.00, 0.10, 0.03, 0.21, 0.00],
       [0.22, 0.66, 0.40, 0.56, 0.00, 0.56, 0.04, 0.00],
       [0.18, 0.01, 0.22, 0.52, 0.39, 0.00, 0.15, 0.00],
       [0.24, 0.04, 0.29, 0.59, 0.27, 0.24, 0.00, 0.00],
       [0.93, 0.95, 0.91, 0.98, 0.58, 0.85, 0.19, 0.00]], dtype=float32)

In [ ]:
df = pd.read_csv('/content/コロナデータ　実験４.csv')

In [ ]:
df

,感染者数,飲食店情報閲覧\n（前年同週比を日毎データに）,人流データ 小売店と娯楽施設\n（基準：2020 年 1 月 3 日〜2 月 6 日の 5 週間の曜日別中央値）,食料品店と薬局,公共交通機関,実効再生産数,緊急事態宣言\n（宣言中は１）,天気\n（全国的に半分が雨\nor台風など上陸などがあったら１）,感染者数.1
0,9.0,-4.310739,-1.0,4.0,1.0,0.749,0.0,1.0,4.0
1,14.0,-4.310739,-9.0,-6.0,-10.0,1.241,0.0,1.0,1.0
2,14.0,-4.310739,-2.0,1.0,-1.0,1.915,0.0,1.0,6.0
3,22.0,-4.310739,1.0,2.0,-2.0,2.692,0.0,0.0,8.0
4,34.0,-4.310739,0.0,2.0,-3.0,4.116,0.0,0.0,9.0
...,...,...,...,...,...,...,...,...,...
583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df=df[0:574]

In [ ]:
df

,感染者数,飲食店情報閲覧\n（前年同週比を日毎データに）,人流データ 小売店と娯楽施設\n（基準：2020 年 1 月 3 日〜2 月 6 日の 5 週間の曜日別中央値）,食料品店と薬局,公共交通機関,実効再生産数,緊急事態宣言\n（宣言中は１）,天気\n（全国的に半分が雨\nor台風など上陸などがあったら１）,感染者数.1
0,9.0,-4.310739,-1.0,4.0,1.0,0.749,0.0,1.0,4.0
1,14.0,-4.310739,-9.0,-6.0,-10.0,1.241,0.0,1.0,1.0
2,14.0,-4.310739,-2.0,1.0,-1.0,1.915,0.0,1.0,6.0
3,22.0,-4.310739,1.0,2.0,-2.0,2.692,0.0,0.0,8.0
4,34.0,-4.310739,0.0,2.0,-3.0,4.116,0.0,0.0,9.0
...,...,...,...,...,...,...,...,...,...
569,2219.0,-58.803064,-18.0,3.0,-27.0,0.860,1.0,0.0,8223.0
570,1758.0,-58.803064,-14.0,7.0,-26.0,0.838,1.0,0.0,10585.0
571,3229.0,-58.803064,-18.0,3.0,-28.0,0.815,1.0,1.0,12378.0
572,3598.0,-56.676452,-18.0,3.0,-28.0,0.794,1.0,0.0,10377.0


In [ ]:
df.isnull().sum()

感染者数                                                        0
飲食店情報閲覧\n（前年同週比を日毎データに）                                     0
人流データ　小売店と娯楽施設\n（基準：2020 年 1 月 3 日〜2 月 6 日の 5 週間の曜日別中央値）    0
食料品店と薬局                                                     0
公共交通機関                                                      0
実効再生産数                                                      0
緊急事態宣言\n（宣言中は１）                                             0
天気\n（全国的に半分が雨\nor台風など上陸などがあったら１）                            0
感染者数.1                                                      0
dtype: int64

感染者数を２週間ずらし、２週間前の感染者数も変数に入れる

In [ ]:
# numpyの出力を小数点2桁に
np.set_printoptions(precision=2, floatmode='fixed', suppress=True)

# 因果探索の結果を格納するリスト
m_list = []
loss_list = []

for i in range(5):
    m, loss = run_SAM(in_data=df, lr_gen=0.01*0.5,
                      lr_disc=0.01*0.5*2,
                      #lambda1=0.01, lambda2=1e-05,
                      lambda1=5.0*20, lambda2=0.005*20,
                      hlayers=2,
                      nh=200, dnh=200,
                      train_epochs=10000,
                      test_epochs=1000,
                      device='cuda:0')

    print(loss)
    print(m)

    m_list.append(m)
    loss_list.append(loss)

100%|██████████| 11000/11000 [07:20<00:00, 24.97it/s, disc=1.74, gen=4.38, regul_loss=4.59, tot=54.4]


[[4.61]]
[[0.00 0.00 0.01 0.00 0.00 0.99 0.00 0.00 0.24]
 [0.19 0.00 1.00 0.01 0.10 0.98 1.00 0.00 0.08]
 [0.00 0.07 0.00 0.01 0.00 0.82 0.22 0.01 0.00]
 [0.00 0.54 1.00 0.00 0.96 0.97 0.79 0.00 0.11]
 [0.34 0.98 1.00 0.01 0.00 0.82 0.05 0.00 0.04]
 [0.00 0.02 0.02 0.01 0.02 0.00 0.96 0.00 0.00]
 [0.00 0.01 0.00 0.01 0.04 0.00 0.00 0.00 0.02]
 [1.00 0.98 0.99 0.99 0.96 0.94 0.00 0.00 0.98]
 [1.00 1.00 0.00 0.01 0.34 0.00 0.98 0.02 0.00]]


100%|██████████| 11000/11000 [07:22<00:00, 24.88it/s, disc=0.131, gen=11.3, regul_loss=3.89, tot=111]


[[9.36]]
[[0.00 0.17 0.03 0.00 0.01 1.00 0.00 0.00 0.09]
 [0.00 0.00 0.25 1.00 0.07 0.97 1.00 0.00 0.05]
 [0.01 0.04 0.00 0.10 0.06 0.00 0.00 0.00 0.01]
 [0.01 0.01 0.98 0.00 0.03 0.00 0.00 0.00 0.01]
 [0.01 0.68 1.00 0.94 0.00 0.98 0.12 0.00 0.00]
 [0.03 0.00 0.25 0.99 0.03 0.00 0.15 0.00 0.00]
 [0.08 0.04 0.92 0.99 0.04 0.49 0.00 0.01 0.04]
 [0.97 0.94 0.00 0.99 0.82 0.96 0.00 0.00 0.99]
 [0.97 0.97 0.00 0.99 0.92 0.00 0.29 0.01 0.00]]


100%|██████████| 11000/11000 [07:21<00:00, 24.94it/s, disc=0.0272, gen=8.57, regul_loss=5.64, tot=91.3]


[[10.16]]
[[0.00 0.05 0.00 0.00 0.00 1.00 0.00 0.00 0.05]
 [0.00 0.00 0.97 1.00 0.04 1.00 1.00 0.00 0.99]
 [0.94 0.06 0.00 0.99 0.08 0.65 0.11 0.00 0.98]
 [0.99 0.03 0.15 0.00 0.03 0.97 0.01 0.00 0.87]
 [0.99 0.98 0.96 0.34 0.00 0.99 0.00 0.00 0.00]
 [0.07 0.00 0.01 0.00 0.01 0.00 0.01 0.00 0.03]
 [0.47 0.04 0.04 0.44 0.03 1.00 0.00 0.01 0.97]
 [0.99 0.88 0.88 0.94 0.99 0.99 0.00 0.00 1.00]
 [0.99 0.06 0.00 0.02 0.00 0.00 0.04 0.00 0.00]]


100%|██████████| 11000/11000 [07:23<00:00, 24.79it/s, disc=0.323, gen=7.94, regul_loss=4.06, tot=87.8]


[[8.15]]
[[0.00 0.22 0.98 0.89 0.99 1.00 0.00 0.00 0.99]
 [0.01 0.00 1.00 0.09 0.98 0.74 1.00 0.00 0.05]
 [0.00 0.06 0.00 0.03 0.00 0.00 0.00 0.00 0.02]
 [0.02 0.06 0.99 0.00 0.96 0.00 0.00 0.01 0.04]
 [0.00 0.11 1.00 0.01 0.00 0.33 0.00 0.00 0.01]
 [0.03 0.04 0.00 0.00 0.10 0.00 0.01 0.00 0.00]
 [0.01 0.06 0.00 0.27 0.03 0.99 0.00 0.00 0.03]
 [0.98 0.93 0.98 0.99 0.00 0.96 0.00 0.00 1.00]
 [0.26 0.92 0.00 0.56 0.91 0.00 0.00 0.01 0.00]]


100%|██████████| 11000/11000 [07:28<00:00, 24.53it/s, disc=0.265, gen=11.5, regul_loss=3.19, tot=115]

[[10.13]]
[[0.00 0.00 0.00 0.98 0.00 1.00 0.00 0.00 0.18]
 [0.16 0.00 1.00 0.01 0.01 0.00 1.00 0.00 0.05]
 [0.01 0.04 0.00 0.07 0.02 0.00 0.00 0.00 0.00]
 [0.01 0.01 1.00 0.00 0.31 0.85 0.00 0.00 0.02]
 [0.01 0.26 1.00 0.00 0.00 0.54 0.01 0.00 0.00]
 [0.01 0.03 0.00 0.05 0.00 0.00 0.04 0.00 0.00]
 [0.48 0.04 0.97 0.80 0.46 0.78 0.00 0.01 0.02]
 [0.91 0.95 0.97 0.98 0.00 0.97 0.00 0.00 0.99]
 [0.99 0.97 0.00 0.00 0.98 0.00 0.21 0.03 0.00]]


In [ ]:
m_list1=m_list

In [ ]:
final_list1=sum(m_list1)/len(m_list1)

In [ ]:
final_list1

array([[0.00, 0.09, 0.20, 0.37, 0.20, 1.00, 0.00, 0.00, 0.31],
       [0.07, 0.00, 0.84, 0.42, 0.24, 0.74, 1.00, 0.00, 0.24],
       [0.19, 0.05, 0.00, 0.24, 0.03, 0.29, 0.07, 0.00, 0.20],
       [0.21, 0.13, 0.82, 0.00, 0.46, 0.56, 0.16, 0.00, 0.21],
       [0.27, 0.60, 0.99, 0.26, 0.00, 0.73, 0.04, 0.00, 0.01],
       [0.03, 0.02, 0.05, 0.21, 0.03, 0.00, 0.23, 0.00, 0.01],
       [0.21, 0.04, 0.39, 0.50, 0.12, 0.65, 0.00, 0.01, 0.22],
       [0.97, 0.94, 0.76, 0.98, 0.55, 0.96, 0.00, 0.00, 0.99],
       [0.84, 0.78, 0.00, 0.31, 0.63, 0.00, 0.30, 0.01, 0.00]],
      dtype=float32)

学習率0.01を0.1

In [ ]:
# numpyの出力を小数点2桁に
np.set_printoptions(precision=2, floatmode='fixed', suppress=True)

# 因果探索の結果を格納するリスト
m_list = []
loss_list = []

for i in range(5):
    m, loss = run_SAM(in_data=df, lr_gen=0.1*0.5,
                      lr_disc=0.1*0.5*2,
                      #lambda1=0.01, lambda2=1e-05,
                      lambda1=5.0*20, lambda2=0.005*20,
                      hlayers=2,
                      nh=200, dnh=200,
                      train_epochs=10000,
                      test_epochs=1000,
                      device='cuda:0')

    print(loss)
    print(m)

    m_list.append(m)
    loss_list.append(loss)

100%|██████████| 11000/11000 [07:31<00:00, 24.35it/s, disc=1.55, gen=1.01, regul_loss=3.68, tot=12.9]


[[1.01]]
[[0.00 0.00 1.00 1.00 0.00 1.00 0.00 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.03 0.00 0.00 0.00 0.00]
 [0.00 1.00 0.00 1.00 0.99 1.00 0.00 0.00 0.00]
 [0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.01]
 [0.00 1.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00]
 [0.00 0.00 0.00 0.02 0.00 0.00 0.00 0.00 0.00]
 [1.00 1.00 1.00 1.00 1.00 1.00 0.00 0.00 0.00]
 [0.99 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00]
 [1.00 0.00 0.00 1.00 0.00 0.00 1.00 0.00 0.00]]


100%|██████████| 11000/11000 [07:26<00:00, 24.62it/s, disc=0.174, gen=8.86, regul_loss=3.16, tot=86.2]


[[8.67]]
[[0.00 0.00 0.00 1.00 0.00 1.00 1.00 0.00 0.13]
 [0.00 0.00 1.00 1.00 0.01 0.00 0.00 0.00 0.02]
 [0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00]
 [0.00 0.00 0.07 0.00 0.00 0.00 0.00 0.00 0.00]
 [0.00 1.00 1.00 0.00 0.00 0.99 0.00 0.00 1.00]
 [0.07 1.00 0.00 0.00 0.00 0.00 1.00 0.62 0.00]
 [0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.01]
 [1.00 1.00 0.00 1.00 0.00 0.00 1.00 1.00 0.00]]


100%|██████████| 11000/11000 [07:28<00:00, 24.54it/s, disc=0.0773, gen=10.8, regul_loss=3.5, tot=102]


[[11.02]]
[[0.00 0.00 0.00 0.00 0.07 0.00 1.00 0.00 1.00]
 [0.00 0.00 0.00 0.04 1.00 0.33 0.00 0.00 0.00]
 [0.00 1.00 0.00 1.00 0.00 0.00 0.19 0.00 1.00]
 [1.00 1.00 0.00 0.00 1.00 1.00 0.00 0.00 0.00]
 [0.00 0.03 0.00 0.02 0.00 0.00 0.00 0.00 0.00]
 [0.00 0.00 0.00 0.03 0.00 0.00 0.00 0.00 0.00]
 [0.00 1.00 0.00 0.00 0.00 1.00 0.00 1.00 1.00]
 [0.00 1.00 0.00 0.00 0.00 1.00 0.00 0.00 1.00]
 [0.00 1.00 0.00 0.07 0.00 1.00 0.00 0.00 0.00]]


100%|██████████| 11000/11000 [07:37<00:00, 24.07it/s, disc=0.689, gen=7, regul_loss=2.98, tot=66]


[[7.02]]
[[0.00 0.00 0.00 1.00 0.00 0.00 1.00 0.00 1.00]
 [0.00 0.00 0.00 0.96 1.00 1.00 1.00 0.00 1.00]
 [0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00]
 [0.00 0.00 0.00 0.00 1.00 0.00 1.00 0.00 1.00]
 [0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00 1.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00]]


100%|██████████| 11000/11000 [07:34<00:00, 24.20it/s, disc=0.217, gen=8.73, regul_loss=3.68, tot=87.5]

[[8.40]]
[[0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.06]
 [0.17 0.00 1.00 1.00 0.99 1.00 0.00 0.00 0.07]
 [0.00 0.03 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
 [0.00 0.00 1.00 0.00 1.00 0.00 0.16 0.00 0.00]
 [0.00 0.06 1.00 0.03 0.00 0.00 0.06 0.00 0.00]
 [1.00 0.00 0.00 0.00 0.00 0.00 0.62 0.00 0.00]
 [0.00 0.18 0.00 0.00 1.00 0.00 0.00 0.00 0.01]
 [0.00 0.97 1.00 1.00 0.00 1.00 0.54 0.00 0.99]
 [1.00 0.99 0.00 0.00 1.00 0.00 1.00 0.00 0.00]]


In [ ]:
m_list2=m_list

In [ ]:
final_list2=sum(m_list2)/len(m_list2)

In [ ]:
final_list2

array([[0.00, 0.00, 0.20, 0.60, 0.01, 0.40, 0.80, 0.00, 0.44],
       [0.03, 0.00, 0.40, 0.60, 0.61, 0.47, 0.20, 0.00, 0.22],
       [0.00, 0.41, 0.00, 0.60, 0.20, 0.40, 0.04, 0.00, 0.20],
       [0.20, 0.40, 0.21, 0.00, 0.60, 0.20, 0.23, 0.00, 0.20],
       [0.00, 0.42, 0.40, 0.01, 0.00, 0.60, 0.01, 0.00, 0.40],
       [0.21, 0.20, 0.00, 0.01, 0.00, 0.00, 0.52, 0.12, 0.00],
       [0.20, 0.64, 0.20, 0.20, 0.40, 0.40, 0.00, 0.40, 0.20],
       [0.20, 0.39, 0.20, 0.40, 0.00, 0.40, 0.11, 0.00, 0.40],
       [0.60, 0.60, 0.00, 0.41, 0.20, 0.20, 0.80, 0.20, 0.00]],
      dtype=float32)

学習率0.5を0.05

In [ ]:
# numpyの出力を小数点2桁に
np.set_printoptions(precision=2, floatmode='fixed', suppress=True)

# 因果探索の結果を格納するリスト
m_list = []
loss_list = []

for i in range(5):
    m, loss = run_SAM(in_data=df, lr_gen=0.01*0.05,
                      lr_disc=0.01*0.05*2,
                      #lambda1=0.01, lambda2=1e-05,
                      lambda1=5.0*20, lambda2=0.005*20,
                      hlayers=2,
                      nh=200, dnh=200,
                      train_epochs=10000,
                      test_epochs=1000,
                      device='cuda:0')

    print(loss)
    print(m)

    m_list.append(m)
    loss_list.append(loss)

100%|██████████| 11000/11000 [07:38<00:00, 23.99it/s, disc=0.0675, gen=6.9, regul_loss=3.16, tot=97.9]


[[7.89]]
[[0.00 0.33 0.26 0.26 0.16 0.22 0.25 0.10 0.29]
 [0.21 0.00 0.51 0.30 0.26 0.36 0.51 0.08 0.32]
 [0.23 0.33 0.00 0.44 0.39 0.29 0.25 0.15 0.25]
 [0.26 0.25 0.33 0.00 0.23 0.20 0.20 0.08 0.20]
 [0.23 0.39 0.44 0.28 0.00 0.45 0.28 0.09 0.30]
 [0.31 0.09 0.12 0.21 0.13 0.00 0.23 0.07 0.21]
 [0.26 0.29 0.20 0.25 0.22 0.27 0.00 0.09 0.26]
 [0.20 0.18 0.15 0.23 0.15 0.29 0.16 0.00 0.30]
 [0.57 0.28 0.23 0.22 0.13 0.24 0.30 0.08 0.00]]


100%|██████████| 11000/11000 [07:34<00:00, 24.22it/s, disc=0.0864, gen=6.95, regul_loss=2.64, tot=94.7]


[[6.65]]
[[0.00 0.31 0.27 0.23 0.17 0.27 0.23 0.09 0.30]
 [0.23 0.00 0.50 0.34 0.27 0.35 0.59 0.08 0.26]
 [0.26 0.29 0.00 0.34 0.35 0.27 0.27 0.11 0.22]
 [0.27 0.27 0.35 0.00 0.23 0.25 0.24 0.07 0.27]
 [0.22 0.41 0.37 0.26 0.00 0.54 0.27 0.10 0.26]
 [0.30 0.12 0.17 0.19 0.13 0.00 0.23 0.07 0.14]
 [0.28 0.23 0.22 0.24 0.21 0.26 0.00 0.08 0.27]
 [0.18 0.13 0.17 0.33 0.16 0.25 0.23 0.00 0.43]
 [0.52 0.27 0.24 0.19 0.16 0.33 0.27 0.07 0.00]]


100%|██████████| 11000/11000 [07:38<00:00, 24.01it/s, disc=0.132, gen=5.61, regul_loss=3.51, tot=79.7]


[[6.15]]
[[0.00 0.30 0.20 0.23 0.16 0.29 0.19 0.08 0.28]
 [0.17 0.00 0.51 0.30 0.31 0.34 0.70 0.08 0.25]
 [0.22 0.33 0.00 0.39 0.46 0.36 0.27 0.09 0.19]
 [0.24 0.25 0.30 0.00 0.23 0.26 0.23 0.07 0.24]
 [0.21 0.31 0.34 0.24 0.00 0.44 0.21 0.08 0.22]
 [0.23 0.10 0.13 0.21 0.14 0.00 0.23 0.08 0.17]
 [0.25 0.16 0.16 0.20 0.21 0.24 0.00 0.07 0.25]
 [0.21 0.11 0.22 0.24 0.13 0.22 0.19 0.00 0.39]
 [0.58 0.25 0.17 0.19 0.12 0.31 0.23 0.07 0.00]]


100%|██████████| 11000/11000 [07:41<00:00, 23.82it/s, disc=0.0903, gen=6.49, regul_loss=3.86, tot=95.9]


[[7.07]]
[[0.00 0.32 0.24 0.24 0.22 0.28 0.26 0.12 0.40]
 [0.22 0.00 0.49 0.32 0.26 0.35 0.56 0.09 0.31]
 [0.26 0.36 0.00 0.41 0.38 0.30 0.28 0.15 0.24]
 [0.25 0.26 0.30 0.00 0.25 0.26 0.23 0.12 0.25]
 [0.25 0.39 0.42 0.29 0.00 0.49 0.27 0.11 0.29]
 [0.29 0.11 0.18 0.21 0.16 0.00 0.27 0.09 0.18]
 [0.26 0.22 0.22 0.26 0.23 0.26 0.00 0.08 0.26]
 [0.22 0.14 0.15 0.25 0.16 0.26 0.22 0.00 0.27]
 [0.42 0.23 0.21 0.22 0.13 0.28 0.31 0.12 0.00]]


100%|██████████| 11000/11000 [07:40<00:00, 23.89it/s, disc=0.0577, gen=7.39, regul_loss=3.5, tot=102]

[[7.40]]
[[0.00 0.29 0.24 0.23 0.18 0.23 0.17 0.13 0.43]
 [0.20 0.00 0.56 0.31 0.32 0.34 0.79 0.09 0.38]
 [0.24 0.37 0.00 0.54 0.45 0.27 0.22 0.13 0.30]
 [0.22 0.23 0.26 0.00 0.24 0.23 0.24 0.11 0.25]
 [0.19 0.32 0.34 0.27 0.00 0.47 0.19 0.11 0.28]
 [0.31 0.08 0.17 0.20 0.10 0.00 0.09 0.08 0.19]
 [0.24 0.15 0.20 0.22 0.20 0.33 0.00 0.08 0.28]
 [0.12 0.14 0.14 0.24 0.14 0.18 0.14 0.00 0.29]
 [0.42 0.22 0.18 0.24 0.13 0.28 0.22 0.10 0.00]]
